### Biblioteki

In [ ]:
import pandas as pd
from tqdm import tqdm
from collections import Counter
import re
import unicodedata
import numpy as np


from sentence_transformers import SentenceTransformer
import hdbscan

In [ ]:
from src.data.utils import create_dataframe_from_csv_zst, compress_dataframe_to_zst

### Oferty

In [2]:
LISTINGS_PATH = "../data/raw/v2/listings.csv.zst"

In [4]:
listings_df = create_dataframe_from_csv_zst(LISTINGS_PATH)
listings_df.head()

/tmp/ipykernel_5161/3230345688.py:6: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,availability_eoy,number_of_reviews_ly,estimated_occupancy_l365d,estimated_revenue_l365d
0,27262,https://www.nocarz.pl/rooms/27262,20241225065837,2024-12-25,city scrape,"AQA-No7, Great mattress, high speed internet",THE MATTRESS - KING KOIL - Camden Luxury 160x2...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,37177,...,t,1,1,0,0,0.19,NaN,NaN,NaN,NaN
1,809874,https://www.nocarz.pl/rooms/809874,20241225065837,2024-12-25,city scrape,Wonderfull Penthouse!!,NaN,NaN,https://a0.muscache.com/pictures/11813063/7d06...,4259738,...,f,1,1,0,0,0.68,NaN,NaN,NaN,NaN
2,866381,https://www.nocarz.pl/rooms/866381,20241225065837,2024-12-25,city scrape,Acropolis View Funky House,Welcome to a colorful 7th-floor penthouse in N...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,4551671,...,f,1,1,0,0,0.28,NaN,NaN,NaN,NaN
3,886724,https://www.nocarz.pl/rooms/886724,20241225065837,2024-12-25,city scrape,Luxury Boutique Appartment -Athens,NaN,The apartment is very conveniently located for...,https://a0.muscache.com/pictures/14659524/b33a...,4700824,...,f,1,1,0,0,0.14,NaN,NaN,NaN,NaN
4,896212,https://www.nocarz.pl/rooms/896212,20241225065837,2024-12-25,city scrape,Living like in a cottage in the center of Athens,An oasis of calm in the centre of a crowded ci...,Exarchia is considered the bohemian and altern...,https://a0.muscache.com/pictures/prohost-api/H...,4777984,...,t,13,13,0,0,3.58,NaN,NaN,NaN,NaN


### Wyposażenie

In [5]:
amenities = listings_df[['id','amenities']]
amenities.head()

,id,amenities
0,27262,"[""Heating - split type ductless system"", ""Dish..."
1,809874,"[""Host greets you"", ""Dishes and silverware"", ""..."
2,866381,"[""Coffee"", ""Dishes and silverware"", ""Washer"", ..."
3,886724,"[""Host greets you"", ""Luggage dropoff allowed"",..."
4,896212,"[""Host greets you"", ""Coffee"", ""Dishes and silv..."


In [6]:
random_amenities = amenities.sample(10)

for index, row in random_amenities.iterrows():
    print(f"ID: {row['id']}")
    print(row['amenities'])
    print()


ID: 28624466
["Smoke alarm", "Pack \u2019n play/Travel crib", "Self check-in", "Baby bath", "Pocket wifi", "Air conditioning", "Iron", "Wifi", "Cooking basics", "Stove", "Shampoo", "Refrigerator", "Window guards", "Oven", "Hair dryer", "TV", "Dishes and silverware", "Hot water", "Room-darkening shades", "Exterior security cameras on property", "Elevator", "Dryer", "Fire extinguisher", "Hangers", "Kitchen", "Keypad", "Carbon monoxide alarm", "First aid kit", "Coffee maker", "High chair", "Long term stays allowed", "Private entrance", "Extra pillows and blankets", "Essentials", "Bed linens", "Microwave", "Washer", "Heating"]

ID: 852229300980580833
["Air conditioning", "Kitchen", "Dishes and silverware", "TV", "Refrigerator", "Fire extinguisher", "Heating", "Wifi", "Coffee maker"]

ID: 1015987829563230965
["Portable fans", "Bidet", "Single level home", "Paid parking off premises", "Iron", "Wifi", "Cooking basics", "Dining table", "Stove", "Shampoo", "Refrigerator", "Oven", "Body soap", "

In [7]:
all_features = []
for index, row in tqdm(amenities.iterrows(), total=amenities.shape[0]):
    try:
        amenities_list = eval(row['amenities'])
        all_features.extend(amenities_list)
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        continue

print(len(all_features))

100%|██████████| 39930/39930 [00:04<00:00, 8874.17it/s] 

1135918


In [12]:
feature_counts = Counter(all_features)
print(f"Unique features: {len(feature_counts)}")

most_common_features = feature_counts.most_common(10)
least_common_features = feature_counts.most_common()[:-11:-1]

print("Most common features:")
for feature, count in most_common_features:
    print(f"{feature}: {count}")

print("\nLeast common features:")
for feature, count in least_common_features:
    print(f"{feature}: {count}")

Unique features: 5791
Most common features:
Wifi: 36424
Kitchen: 35645
Hot water: 31345
Hair dryer: 30332
Hangers: 29912
Essentials: 29397
Iron: 29170
Dishes and silverware: 28457
Bed linens: 27949
Cooking basics: 26346

Least common features:
TV with Apple TV, HBO Max, Netflix: 1
Marca blanca refrigerator: 1
30 inch HDTV with Amazon Prime Video, Apple TV, Disney+, HBO Max, Netflix: 1
Housekeeping available from 11:30 AM to 4:00 PM, every day - available at extra cost: 1
Housekeeping available from 8:00 AM to 6:30 PM, every day - available at extra cost: 1
HDTV with Disney+, Apple TV, HBO Max: 1
75 inch HDTV with Amazon Prime Video, HBO Max: 1
Nívea body soap: 1
Bed Head shampoo: 1
CREATE  refrigerator: 1


In [13]:
#looking for words similar to wifi
similar_words = []
for feature in feature_counts:
    if 'wifi' in feature:
        similar_words.append(feature)
print("Words similar to 'wifi':")
for word in similar_words:
    print(word)

Words similar to 'wifi':
Fast wifi – 168 Mbps
Pocket wifi
Fast wifi – 100 Mbps
Fast wifi – 93 Mbps
Fast wifi – 87 Mbps
Fast wifi – 303 Mbps
Fast wifi – 197 Mbps
Fast wifi – 196 Mbps
Fast wifi – 99 Mbps
Fast wifi – 464 Mbps
Fast wifi – 204 Mbps
Fast wifi – 203 Mbps
Fast wifi – 716 Mbps
Sonos - wifi Bluetooth sound system
Fast wifi – 253 Mbps
Fast wifi – 75 Mbps
Fast wifi – 52 Mbps
Fast wifi – 90 Mbps
Fast wifi – 94 Mbps
Fast wifi – 263 Mbps
Fast wifi – 487 Mbps
Fast wifi – 97 Mbps
Fast wifi – 112 Mbps
Fast wifi – 138 Mbps
Fast wifi – 120 Mbps
Fast wifi – 144 Mbps
Fast wifi – 179 Mbps
Fast wifi – 166 Mbps
Fast wifi – 80 Mbps
Fast wifi – 101 Mbps
Fast wifi – 427 Mbps
Fast wifi – 84 Mbps
Fast wifi – 98 Mbps
Fast wifi – 60 Mbps
Fast wifi – 91 Mbps
Fast wifi – 82 Mbps
Fast wifi – 468 Mbps
Fast wifi – 103 Mbps
Fast wifi – 77 Mbps
Fast wifi – 95 Mbps
Fast wifi – 54 Mbps
Fast wifi – 206 Mbps
Fast wifi – 104 Mbps
Fast wifi – 195 Mbps
Fast wifi – 208 Mbps
Fast wifi – 51 Mbps
Fast wifi – 148 Mbps


In [16]:
display(feature_counts['Wifi'])
display(feature_counts['Fast wifi – 303 Mbps'])

36424

6

- Widać, że wiele nazw wyposażenia, jest nazywane w podbny sposób, aczkolwiek nie jest to ustandaryzowane
- Ustandaryzowanie nazewnictwa, mogłoby pomóc w ustaleniu, czy wyposażenie wpływa na długość pobytu

#### Standaryzacja wyposażenia

In [18]:
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
model._first_module()

Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 

In [67]:
def clean_text(text):
    if not isinstance(text, str):
        return None

    # text preprocessing
    text = text.strip()
    text = text.replace('\n', ' ').replace('\t', ' ')

    text = ''.join(ch for ch in text if unicodedata.category(ch)[0] != 'C')

    text = re.sub(r'\s+', ' ', text)

    if len(text) < 2:
        return None

    return text

# only keeping features that we are able to encode
unique_features = []
for f in feature_counts.keys():
    clean_f = clean_text(f)
    if clean_f is None:
        continue
    try:
        model.encode([clean_f], device='cuda')
        unique_features.append(clean_f)
    except:
        print(f"Pominięto: {repr(f)}")

print(f"Clean records remaining :  {len(unique_features)}")

Clean records remaining :  5791


In [ ]:
embeddings = model.encode(unique_features, device='cuda', show_progress_bar=True)
cluster = hdbscan.HDBSCAN(min_cluster_size=2)
labels = cluster.fit_predict(embeddings)

df = pd.DataFrame({
    'feature': unique_features,
    'label': labels,
    'count': [feature_counts[feature] for feature in unique_features]
})

normalized_map = {}

for label in set(labels):
    if label == -1:
        continue
    group = df[df["label"] == label]

    repr_name = group.sort_values("count", ascending=False).iloc[0]["feature"]
    for item in group["feature"]:
        normalized_map[item] = repr_name

# Map unclustered features to "OTHER"
for f in df[df["label"] == -1]["feature"]:
    normalized_map[f] = "OTHER"

print("Normalized map:")
for k, v in normalized_map.items():
    print(f"{k}: {v}")


Batches:   0%|          | 0/181 [00:00<?, ?it/s]

/root/.cache/pypoetry/virtualenvs/ium-long-stay-insights-DB1IbSqe-py3.12/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/root/.cache/pypoetry/virtualenvs/ium-long-stay-insights-DB1IbSqe-py3.12/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Normalized map:
Keypad: Lockbox
Lockbox: Lockbox
Smart lock: Lockbox
Lock on bedroom door: Lockbox
Changing table - available upon request: Changing table
Changing table: Changing table
Changing table - always at the listing: Changing table
Board games: Board games
Game console: Board games
Game console: PS4: Board games
Life size games: Board games
Arcade games: Board games
Game console: PS3: Board games
Game console: Xbox 360: Board games
Game console: Nintendo Wii and Nintendo Wii U: Board games
Game console: Xbox One: Board games
Game console: PS5: Board games
Game console: PS2: Board games
Game console: Nintendo Switch: Board games
Game console: Nintendo Switch and Xbox 360: Board games
Game console: Xbox Series X: Board games
Game console: Nintendo Wii and PS3: Board games
Game console: Nintendo Wii: Board games
Game console: Nintendo Switch and PS5: Board games
Game console: PS2 and Xbox 360: Board games
Ironing board: Board games
Free carport on premises – 1 space: Free carport

In [46]:
unique_normalized_features = set(normalized_map.values())
print(f"Unique normalized features: {len(unique_normalized_features)}")

Unique normalized features: 463


In [ ]:
most_popular_normalized_features = Counter(normalized_map.values()).most_common(10)
print("Most popular normalized features:")
for feature, count in most_popular_normalized_features:
    print(f"{feature}: {count}")

print(feature_counts.most_common(10))

Most popular normalized features:
OTHER: 1533
TV: 1277
Wifi: 580
Shared outdoor pool - available seasonally, open specific hours: 77
Housekeeping available from 11:00 AM to 3:00 PM, every day - available at extra cost: 64
Coffee maker: 48
Exercise equipment: 47
Housekeeping available from 11:00 AM to 2:30 PM, Tuesday, Friday - available at extra cost: 27
Housekeeping available from 11:00 AM to 5:00 PM, 2 days a week - available at extra cost: 25
Board games: 19
[('Wifi', 36424), ('Kitchen', 35645), ('Hot water', 31345), ('Hair dryer', 30332), ('Hangers', 29912), ('Essentials', 29397), ('Iron', 29170), ('Dishes and silverware', 28457), ('Bed linens', 27949), ('Cooking basics', 26346)]


In [63]:
amenities.loc[:, 'standardized_amenities'] = amenities['amenities'].apply(lambda x: [normalized_map.get(f, "OTHER") for f in eval(x)])

In [64]:
amenities.head()
all_std_amenities = []

for index, row in amenities.iterrows():
    all_std_amenities.extend(row['standardized_amenities'])
print(f"Amount of all standarized amenities: {len(all_std_amenities)}")

print(f"Amount of unique standarized amenities: {len(set(all_std_amenities))}")

Amount of all standarized amenities: 1135918
Amount of unique standarized amenities: 453


In [70]:
std_amenitis_counter = Counter(all_std_amenities)
print("Most common standardized amenities:\n")
for amenity, count in std_amenitis_counter.most_common(10):
    print(f"{amenity}: {count}")


print("\nLeast common standardized amenities: \n")
for amenity, count in std_amenitis_counter.most_common()[:-11:-1]:
    print(f"{amenity}: {count}")

Most common standardized amenities:

OTHER: 730768
Wifi: 40330
Coffee maker: 36893
TV: 34669
Drying rack for clothing: 34613
Washer: 30639
Heating: 27894
Air conditioning: 27237
Refrigerator: 25898
Shampoo: 23813

Least common standardized amenities: 

Apple Homepod Bluetooth sound system: 1
Farmacia body soap: 1
Papoutsanis καραβακι shampoo: 1
Papoutsanis καραβακι body soap: 1
Culaquiera body soap: 2
Todo tipo de cabellos shampoo: 2
Corbero stainless steel oven: 2
Fagor induction stove: 2
Sebamed, eucerin (hipoalergénicos) body soap: 2
Gel de baño y champú body soap: 2


- Po standaryzacji widać, że do hasła wifi zostało przypisanych 580 podobno brzmiących haseł
- Zwiększyły się liczby udogodnień takich jak Wifi, TV..
- Dużo pojedyńczych wystąpień zostało przypisanych do OTHER - można będzie to zliczyć, jako ilość dodatkowych udogodnień
- Dalej można używać, zwykłej listy w celu wybrania udogodnień takich jak "Kitchen"
- Standaryzację, można uznać za pomyślną

In [66]:
amenities.head()

,id,amenities,standardized_amenities
0,27262,"[""Heating - split type ductless system"", ""Dish...","[OTHER, OTHER, OTHER, OTHER, Refrigerator, OTH..."
1,809874,"[""Host greets you"", ""Dishes and silverware"", ""...","[OTHER, OTHER, Washer, TV, OTHER, Refrigerator..."
2,866381,"[""Coffee"", ""Dishes and silverware"", ""Washer"", ...","[Coffee maker, OTHER, Washer, Conditioner, OTH..."
3,886724,"[""Host greets you"", ""Luggage dropoff allowed"",...","[OTHER, OTHER, Washer, OTHER, OTHER, Heating, ..."
4,896212,"[""Host greets you"", ""Coffee"", ""Dishes and silv...","[OTHER, Coffee maker, OTHER, OTHER, OTHER, OTH..."


### Statystyki wyposażenia

In [89]:
top_10_most_common = [amenity for amenity in std_amenitis_counter.most_common(11)[1:11] if amenity[0] != "OTHER"]
amenities.loc[:, 'num_of_top_10_common_amenities'] = amenities['standardized_amenities'].apply(
    lambda x: sum(x.count(amenity) for amenity, _ in top_10_most_common)
)
amenities.loc[:, 'num_of_amenities'] = amenities['standardized_amenities'].apply(lambda x: len(x))
amenities.loc[:, 'num_of_other_amenities'] = amenities['standardized_amenities'].apply(lambda x: x.count("OTHER"))
amenities.loc[:, 'has_wifi'] = amenities['standardized_amenities'].apply(lambda x: True if 'Wifi' in x else False)
amenities.loc[:, 'has_air_conditioning'] = amenities['standardized_amenities'].apply(lambda x: True if 'Air conditioning' in x else False)

In [90]:
amenities.head()

,id,amenities,standardized_amenities,num_of_amenities,num_of_other_amenities,has_wifi,has_air_conditioning,num_of_top_10_common_amenities
0,27262,"[""Heating - split type ductless system"", ""Dish...","[OTHER, OTHER, OTHER, OTHER, Refrigerator, OTH...",37,25,True,False,9
1,809874,"[""Host greets you"", ""Dishes and silverware"", ""...","[OTHER, OTHER, Washer, TV, OTHER, Refrigerator...",21,12,True,True,7
2,866381,"[""Coffee"", ""Dishes and silverware"", ""Washer"", ...","[Coffee maker, OTHER, Washer, Conditioner, OTH...",49,29,True,True,12
3,886724,"[""Host greets you"", ""Luggage dropoff allowed"",...","[OTHER, OTHER, Washer, OTHER, OTHER, Heating, ...",20,13,True,True,6
4,896212,"[""Host greets you"", ""Coffee"", ""Dishes and silv...","[OTHER, Coffee maker, OTHER, OTHER, OTHER, OTH...",45,31,True,True,9


In [78]:
offers_without_wifi = amenities[amenities['has_wifi'] == False]
print(f"Number of offers without wifi: {len(offers_without_wifi)}")

offers_without_air_conditioning = amenities[amenities['has_air_conditioning'] == False]
print(f"Number of offers without air conditioning: {len(offers_without_air_conditioning)}")

Number of offers without wifi: 993
Number of offers without air conditioning: 12759


### Średnia embeddingów

In [ ]:
def get_mean_embedding(amenities_list, model):
    if not amenities_list:
        return np.zeros(model.get_sentence_embedding_dimension())
    embeddings = model.encode(amenities_list, show_progress_bar=False)
    return np.mean(embeddings, axis=0)

In [93]:
tqdm.pandas(desc="Calculating mean embeddings")
amenities['mean_embedding'] = amenities['standardized_amenities'].progress_apply(lambda x: get_mean_embedding(x, model))

Calculating mean embeddings: 100%|██████████| 39930/39930 [14:55<00:00, 44.60it/s] 
/tmp/ipykernel_5161/254747570.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amenities['mean_embedding'] = amenities['standardized_amenities'].progress_apply(lambda x: get_mean_embedding(x, model))


In [97]:
amenities['mean_embedding'][0].shape

(384,)

#### Zapisanie do pliku

In [100]:
compress_dataframe_to_zst(amenities, "../data/processed/amenities_stats.csv.zst", compression_level=3)

Compressed to file ../data/processed/amenities_stats.csv.zst with size 96126127 bytes


In [101]:
test_df = create_dataframe_from_csv_zst("../data/processed/amenities_stats.csv.zst")
test_df.head()

,id,amenities,standardized_amenities,num_of_amenities,num_of_other_amenities,has_wifi,has_air_conditioning,num_of_top_10_common_amenities,mean_embedding
0,27262,"[""Heating - split type ductless system"", ""Dish...","['OTHER', 'OTHER', 'OTHER', 'OTHER', 'Refriger...",37,25,True,False,9,[-6.15361333e-02 -5.01407236e-02 -3.76954190e-...
1,809874,"[""Host greets you"", ""Dishes and silverware"", ""...","['OTHER', 'OTHER', 'Washer', 'TV', 'OTHER', 'R...",21,12,True,True,7,[-5.95615655e-02 -3.39257978e-02 -3.55266146e-...
2,866381,"[""Coffee"", ""Dishes and silverware"", ""Washer"", ...","['Coffee maker', 'OTHER', 'Washer', 'Condition...",49,29,True,True,12,[-5.62126637e-02 -3.99859361e-02 -3.53127755e-...
3,886724,"[""Host greets you"", ""Luggage dropoff allowed"",...","['OTHER', 'OTHER', 'Washer', 'OTHER', 'OTHER',...",20,13,True,True,6,[-5.84196560e-02 -4.62852195e-02 -3.04865874e-...
4,896212,"[""Host greets you"", ""Coffee"", ""Dishes and silv...","['OTHER', 'Coffee maker', 'OTHER', 'OTHER', 'O...",45,31,True,True,9,[-5.81393391e-02 -4.77810428e-02 -4.26256545e-...
